In [1]:
import collections
import string
import itertools

In [2]:
Link = collections.namedtuple('Link', 'height left right')

In [3]:
def link_ends(link):
    return set((link.left, link.right))

In [19]:
def read_net(net_string):
    return [Link(h, l, r) for h, (l, r) in enumerate(extract_pairs(net_string))]

In [7]:
def follow(initial_line, links):
    line = initial_line
    heights = sorted(set(l.height for l in links))
    for h in heights:
        for l in [l for l in links if l.height == h]:
            if line in link_ends(l):
                line = [e for e in link_ends(l) if e != line][0]
#                 print(l, line)
    return line

In [9]:
def pack(net):
    packed_links = []
    line_heights = collections.defaultdict(lambda: -1)
    for link in sorted(net):
        link_height = max(line_heights[link.left], line_heights[link.right]) + 1
        line_heights[link.left] = link_height
        line_heights[link.right] = link_height
        packed_links += [Link(link_height, link.left, link.right)]
    return sorted(packed_links)

In [10]:
def follow_many_slow(in_sequence, links):
    out_sequence = [(follow(i, links), term) 
                    for i, term in enumerate(in_sequence)]
    return [term for i, term in sorted(out_sequence)]

In [11]:
def follow_many(in_sequence, net):
    height_groups = [list(g) for _, g in itertools.groupby(pack(net), lambda l: l.height)]
    seq = list(in_sequence)
    for links in height_groups:
        for link in links:
            seq[link.right], seq[link.left] = seq[link.left], seq[link.right]
    return seq

In [12]:
%%timeit
follow_many('abcdefghij', net)

10000 loops, best of 3: 47.5 µs per loop


In [13]:
# %%timeit
# follow_many_slow('abcdefghij', net)

In [14]:
def show_net(links, randomise=False):
    if randomise:
        output = []
        heights = sorted(set(l.height for l in links))
        for h in heights:
            ls = [l for l in links if l.height == h]
            random.shuffle(ls)
            output += ['({}, {})'.format(l.left, l.right) for l in ls]
        return ', '.join(output)
    return ', '.join('({}, {})'.format(l.left, l.right) for l in sorted(links))

In [20]:
def extract_pairs(net_string):
    return [[int(pi) for pi in p.split(', ')] for p in net_string[1:-1].split('), (')]

In [25]:
rrnet = read_net(show_net(net, randomise=True))
rnet = read_net(show_net(net))
rnet == rrnet, pack(rrnet) == pnet

(True, True)

In [26]:
lnet = make_net(10207, 26, 100000)
plnet = pack(lnet)
assert follow_many(string.ascii_lowercase, lnet) == follow_many(string.ascii_lowercase, plnet)
# for i in range(204):
#     assert follow(i, lnet) == follow(i, plnet)

In [27]:
rlnet = read_net(show_net(lnet))
prlnet = pack(rlnet)

In [28]:
max(link.height for link in plnet)

2251

In [29]:
max(link.height for link in lnet)

99998

In [30]:
max(link.height for link in rlnet)

10206

In [31]:
max(link.height for link in prlnet)

2251

In [32]:
assert follow_many(string.ascii_lowercase, lnet) == follow_many(string.ascii_lowercase, prlnet)

In [33]:
%%timeit
follow_many(string.ascii_lowercase, lnet)

10 loops, best of 3: 24.5 ms per loop


In [34]:
# %%timeit
# follow_many_slow(string.ascii_lowercase, lnet)

In [35]:
def eliminable_pairs_slow(net):
    eps = []
    for l in net:
        o = Link(l.height + 1, l.left, l.right)
        if o in net:
            eps += [(l, o)]
    return eps    

In [36]:
def eliminable_pairs(net):
    height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
    eps = []
    for h in range(1, max(height_groups.keys())):
        for l in height_groups[h]:
            o = Link(l.height - 1, l.left, l.right)
            if o in height_groups[h-1]:
                eps += [(l, o)]
    return eps

In [37]:
%%timeit
eliminable_pairs(plnet)

10 loops, best of 3: 23.5 ms per loop


In [38]:
%%timeit
eliminable_pairs_slow(plnet)

1 loop, best of 3: 2.33 s per loop


In [39]:
assert sorted(sorted(p) for p in eliminable_pairs(plnet)) == sorted(sorted(p) for p in eliminable_pairs_slow(plnet))

In [40]:
def eliminable_pair(net):
    for l in net:
        o = Link(l.height + 1, l.left, l.right)
        if o in net:
            return l, o
    return None

In [41]:
def eliminable_pair_hg(height_groups):
    for h in range(1, max(height_groups.keys())):
        for l in height_groups[h]:
            o = Link(l.height - 1, l.left, l.right)
            if o in height_groups[h-1]:
                return l, o
    return None

In [42]:
def eliminate_pairs_slow(net):
    eliminable_links = eliminable_pair(net)
    while eliminable_links:
        net = pack(l for l in net if l not in eliminable_links)
        eliminable_links = eliminable_pair(net)
    return net

In [43]:
def eliminate_pairs(net):
    height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
    eliminable_links = eliminable_pair_hg(height_groups)
    while eliminable_links:
        net = pack(l for l in net if l not in eliminable_links)
        height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
        eliminable_links = eliminable_pair_hg(height_groups)
    return net

In [44]:
print(len(plnet))
elnet = eliminate_pairs_slow(plnet)
len(plnet), len(elnet)

10207


(10207, 9811)

In [45]:
print(len(plnet))
elnet = eliminate_pairs(plnet)
len(plnet), len(elnet)

10207


(10207, 9811)

In [46]:
eliminable_pairs(elnet)

[]

In [47]:
assert eliminate_pairs_slow(plnet) == eliminate_pairs(plnet)

In [48]:
assert follow_many(string.ascii_lowercase,  lnet) == follow_many(string.ascii_lowercase, elnet)
assert follow_many(string.ascii_lowercase, plnet) == follow_many(string.ascii_lowercase, elnet)

In [49]:
%%timeit
elnet = eliminate_pairs_slow(plnet)

1 loop, best of 3: 3min 30s per loop


In [50]:
%%timeit
elnet = eliminate_pairs(plnet)

1 loop, best of 3: 6.27 s per loop


In [51]:
# for i in range(26):
#     assert follow(i, plnet) == follow(i, elnet)
#     assert follow(i,  lnet) == follow(i, elnet)

In [52]:
# follow(0, plnet), follow(0, elnet), follow(0, lnet)

In [53]:
def triple(net):
    x = None
    y = None
    ts = []
    for a in net:
        bs = [l for l in net if l.height == a.height + 1 
              if l.left == a.right or l.right == a.left]
        for b in bs:
            c = Link(a.height + 2, a.left, a.right)
            if c in net:
                ts += [(a, b, c)]
    return ts

In [54]:
def triple_pair(net):
    ts = []
    for a in net:
        a_ends = link_ends(a)
        bs = [l for l in net if l.height == a.height + 1 
              if link_ends(l) & a_ends]
        if len(bs) == 1:
            b = bs[0]
            lines = set((a.left, a.right, b.left, b.right))
            cs = [l for l in net 
                  if l.height == a.height + 2
                  if link_ends(l) & lines]
            if len(cs) == 1:
                c = Link(a.height + 2, a.left, a.right)
                if c in cs:
                    ds = [l for l in net 
                          if l.height == a.height + 3
                          if link_ends(l) & lines]
                    d = Link(a.height + 3, b.left, b.right)
                    if d in ds:
                        ts += [(a, b, c, d)]
    return ts

In [129]:
def triple_pair_hg(height_groups, debug=False):
    ts = []
    for h in range(3, max(height_groups.keys())):
        for d in height_groups[h]:
            if debug: print('d:', d)
            ch = h - 1
            cs = [l for l in height_groups[ch] if link_ends(l) & link_ends(d)]
            if debug: print('cs:', cs)
            while ch > 2 and not cs:
                ch -= 1
                cs = [l for l in height_groups[ch] if link_ends(l) & link_ends(d)]
                if debug: print('cs:', cs)
            if len(cs) == 1:
                c = cs[0]
                lines = set((d.left, d.right, c.left, c.right))
                if debug: print('c:', '; lines:', lines)
                bs = [l for l in height_groups[ch-1] if link_ends(l) & lines]
                b = Link(ch - 1, d.left, d.right)
                if debug: print('b:', b, '; bs:', bs)
                if len(bs) == 1 and b in bs:
                    ah = b.height - 1
                    als = [l for l in height_groups[ah] if link_ends(l) & link_ends(c)]
                    if debug: print('ah:', ah, '; als:', als)
                    while ah > 0 and not als:
                        ah -= 1
                        als = [l for l in height_groups[ah] if link_ends(l) & link_ends(c)]
                        if debug: print('ah:', ah, '; als:', als)
                    a = Link(ah, c.left, c.right)
                    if debug: print('a:', a)
                    if a in als:
                        if debug: print('adding:', a, b, c, d)
                        ts += [(a, b, c, d)]
    return ts

In [132]:
def eliminate_a_triple_pair_slow(net):
    tps = triple_pair(net)
    print('eatp', tps)

    if tps:
        a, b, c, d = tps[0]
#         x = Link(a.height, b.left, b.right)
#         y = Link(b.height, a.left, a.right)
        x = Link(a.height, b.left, b.right)
        y = Link(b.height, a.left, a.right)
        print('removing', a, b, c, d, '; adding', x, y)
        return pack([l for l in net if l not in [a, b, c, d]] + [x, y])
    return None

In [133]:
def eliminate_a_triple_pair(net):
    height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}

    tps = triple_pair_hg(height_groups)
    print('eatp', tps)
    if tps:
        a, b, c, d = tps[0]
        x = Link(a.height, b.left, b.right)
        y = Link(b.height, a.left, a.right)
        print('removing', a, b, c, d, '; adding', x, y)
        return pack([l for l in net if l not in [a, b, c, d]] + [x, y])
    return None

In [58]:
triple(plnet)

[(Link(height=12, left=0, right=5),
  Link(height=13, left=5, right=11),
  Link(height=14, left=0, right=5)),
 (Link(height=29, left=2, right=18),
  Link(height=30, left=1, right=2),
  Link(height=31, left=2, right=18)),
 (Link(height=40, left=2, right=3),
  Link(height=41, left=3, right=18),
  Link(height=42, left=2, right=3)),
 (Link(height=43, left=1, right=10),
  Link(height=44, left=10, right=20),
  Link(height=45, left=1, right=10)),
 (Link(height=91, left=5, right=8),
  Link(height=92, left=3, right=5),
  Link(height=93, left=5, right=8)),
 (Link(height=91, left=5, right=8),
  Link(height=92, left=8, right=18),
  Link(height=93, left=5, right=8)),
 (Link(height=99, left=13, right=17),
  Link(height=100, left=4, right=13),
  Link(height=101, left=13, right=17)),
 (Link(height=120, left=18, right=20),
  Link(height=121, left=9, right=18),
  Link(height=122, left=18, right=20)),
 (Link(height=147, left=19, right=21),
  Link(height=148, left=7, right=19),
  Link(height=149, left=19,

In [59]:
triple_pair(elnet)

[(Link(height=316, left=8, right=17),
  Link(height=317, left=8, right=21),
  Link(height=318, left=8, right=17),
  Link(height=319, left=8, right=21)),
 (Link(height=867, left=4, right=11),
  Link(height=868, left=5, right=11),
  Link(height=869, left=4, right=11),
  Link(height=870, left=5, right=11))]

In [60]:
height_groups = {h: list(g) for h, g in itertools.groupby(pack(elnet), lambda l: l.height)}
triple_pair_hg(height_groups)

[(Link(height=316, left=8, right=17),
  Link(height=317, left=8, right=21),
  Link(height=318, left=8, right=17),
  Link(height=319, left=8, right=21)),
 (Link(height=867, left=4, right=11),
  Link(height=868, left=5, right=11),
  Link(height=869, left=4, right=11),
  Link(height=870, left=5, right=11))]

In [61]:
[l for l in elnet if l.height >= 1367 if l.height <= 1370 if link_ends(l) & set((6, 16, 19))]

[Link(height=1368, left=13, right=19),
 Link(height=1368, left=14, right=16),
 Link(height=1369, left=3, right=6)]

In [62]:
%%timeit
triple_pair(elnet)

1 loop, best of 3: 21.3 s per loop


In [63]:
%%timeit
height_groups = {h: list(g) for h, g in itertools.groupby(pack(elnet), lambda l: l.height)}
triple_pair_hg(height_groups)

10 loops, best of 3: 105 ms per loop


In [64]:
height_groups = {h: list(g) for h, g in itertools.groupby(pack(elnet), lambda l: l.height)}
assert triple_pair_hg(height_groups) == triple_pair(elnet)

In [65]:
[l for l in elnet if l.height >= 242 if l.height <= 245 ] #if l.left in [13, 17, 20] or l.right in [13, 17, 20]]

[Link(height=242, left=8, right=9),
 Link(height=242, left=15, right=21),
 Link(height=243, left=1, right=9),
 Link(height=243, left=8, right=14),
 Link(height=243, left=11, right=15),
 Link(height=244, left=1, right=18),
 Link(height=244, left=6, right=8),
 Link(height=244, left=9, right=20),
 Link(height=244, left=15, right=25),
 Link(height=245, left=0, right=15),
 Link(height=245, left=1, right=16),
 Link(height=245, left=9, right=12),
 Link(height=245, left=18, right=21)]

In [86]:
def eliminate_triple_pairs_slow(net):
    print(len(net))
    new_net = eliminate_a_triple_pair_slow(net)
    while new_net:
        print(len(net))
        net = new_net
        new_net = eliminate_a_triple_pair_slow(net)
    return net

In [118]:
def eliminate_triple_pairs(net):
    print(len(net))
    new_net = eliminate_a_triple_pair(net)
    while new_net:
        print(len(net))
        net = new_net
        new_net = eliminate_a_triple_pair(net)
    return net

In [134]:
etlnet = eliminate_triple_pairs(elnet)

9811
eatp [(Link(height=316, left=8, right=17), Link(height=317, left=8, right=21), Link(height=318, left=8, right=17), Link(height=319, left=8, right=21)), (Link(height=867, left=4, right=11), Link(height=868, left=5, right=11), Link(height=869, left=4, right=11), Link(height=870, left=5, right=11))]
removing Link(height=316, left=8, right=17) Link(height=317, left=8, right=21) Link(height=318, left=8, right=17) Link(height=319, left=8, right=21) ; adding Link(height=316, left=8, right=21) Link(height=317, left=8, right=17)
9811
eatp [(Link(height=866, left=4, right=11), Link(height=867, left=5, right=11), Link(height=868, left=4, right=11), Link(height=869, left=5, right=11))]
removing Link(height=866, left=4, right=11) Link(height=867, left=5, right=11) Link(height=868, left=4, right=11) Link(height=869, left=5, right=11) ; adding Link(height=866, left=5, right=11) Link(height=867, left=4, right=11)
9809
eatp []


In [112]:
etlnet = eliminate_a_triple_pair(elnet)

eatp [(Link(height=316, left=8, right=17), Link(height=317, left=8, right=21), Link(height=318, left=8, right=17), Link(height=319, left=8, right=21)), (Link(height=867, left=4, right=11), Link(height=868, left=5, right=11), Link(height=869, left=4, right=11), Link(height=870, left=5, right=11))]
removing Link(height=316, left=8, right=17) Link(height=317, left=8, right=21) Link(height=318, left=8, right=17) Link(height=319, left=8, right=21) ; adding Link(height=317, left=8, right=17) Link(height=318, left=8, right=21)


In [135]:
assert follow_many(string.ascii_lowercase, etlnet) == follow_many(string.ascii_lowercase, elnet)

In [136]:
[l for l in elnet if l.height >= 315 if l.height <= 320 if link_ends(l) & set((8, 17, 21))] 

[Link(height=315, left=17, right=20),
 Link(height=316, left=8, right=17),
 Link(height=317, left=8, right=21),
 Link(height=318, left=8, right=17),
 Link(height=319, left=8, right=21),
 Link(height=319, left=14, right=17),
 Link(height=320, left=2, right=8),
 Link(height=320, left=17, right=21)]

In [137]:
nf = [l for l in elnet if l.height >= 315 if l.height <= 320 if link_ends(l) & set((8, 17, 21))]
pnf = pack(nf)
pnf

[Link(height=0, left=17, right=20),
 Link(height=1, left=8, right=17),
 Link(height=2, left=8, right=21),
 Link(height=3, left=8, right=17),
 Link(height=4, left=8, right=21),
 Link(height=4, left=14, right=17),
 Link(height=5, left=2, right=8),
 Link(height=5, left=17, right=21)]

In [138]:
height_groups = {h: list(g) for h, g in itertools.groupby(pack(pnf), lambda l: l.height)}
triple_pair_hg(height_groups, debug=True)

d: Link(height=3, left=8, right=17)
cs: [Link(height=2, left=8, right=21)]
c: ; lines: {8, 17, 21}
b: Link(height=1, left=8, right=17) ; bs: [Link(height=1, left=8, right=17)]
ah: 0 ; als: []
a: Link(height=0, left=8, right=21)
d: Link(height=4, left=8, right=21)
cs: [Link(height=3, left=8, right=17)]
c: ; lines: {8, 17, 21}
b: Link(height=2, left=8, right=21) ; bs: [Link(height=2, left=8, right=21)]
ah: 1 ; als: [Link(height=1, left=8, right=17)]
a: Link(height=1, left=8, right=17)
adding: Link(height=1, left=8, right=17) Link(height=2, left=8, right=21) Link(height=3, left=8, right=17) Link(height=4, left=8, right=21)
d: Link(height=4, left=14, right=17)
cs: [Link(height=3, left=8, right=17)]
c: ; lines: {8, 17, 14}
b: Link(height=2, left=14, right=17) ; bs: [Link(height=2, left=8, right=21)]


[(Link(height=1, left=8, right=17),
  Link(height=2, left=8, right=21),
  Link(height=3, left=8, right=17),
  Link(height=4, left=8, right=21))]

In [139]:
setlnet = eliminate_triple_pairs_slow(elnet)

9811
eatp [(Link(height=316, left=8, right=17), Link(height=317, left=8, right=21), Link(height=318, left=8, right=17), Link(height=319, left=8, right=21)), (Link(height=867, left=4, right=11), Link(height=868, left=5, right=11), Link(height=869, left=4, right=11), Link(height=870, left=5, right=11))]
removing Link(height=316, left=8, right=17) Link(height=317, left=8, right=21) Link(height=318, left=8, right=17) Link(height=319, left=8, right=21) ; adding Link(height=316, left=8, right=21) Link(height=317, left=8, right=17)
9811
eatp [(Link(height=866, left=4, right=11), Link(height=867, left=5, right=11), Link(height=868, left=4, right=11), Link(height=869, left=5, right=11))]
removing Link(height=866, left=4, right=11) Link(height=867, left=5, right=11) Link(height=868, left=4, right=11) Link(height=869, left=5, right=11) ; adding Link(height=866, left=5, right=11) Link(height=867, left=4, right=11)
9809
eatp []


In [140]:
len(setlnet)

9807

In [141]:
assert follow_many(string.ascii_lowercase, etlnet) == follow_many(string.ascii_lowercase, elnet)

In [142]:
''.join(follow_many(string.ascii_lowercase, etlnet))

'zdunvslcfiqywjmkobhxtraegp'

In [143]:
''.join(follow_many(string.ascii_lowercase, setlnet))

'zdunvslcfiqywjmkobhxtraegp'

In [144]:
''.join(follow_many(string.ascii_lowercase, elnet))

'zdunvslcfiqywjmkobhxtraegp'

In [145]:
''.join(follow_many(string.ascii_lowercase, lnet))

'zdunvslcfiqywjmkobhxtraegp'

In [146]:
eliminable_pairs(etlnet)

[]

In [147]:
len(lnet), len(etlnet)

(10207, 9807)

In [148]:
def simplify(net0):
    netp = eliminate_pairs(net0)
    new_net = eliminate_a_triple_pair(netp)
    while new_net:
#         print('sipl', len(net0), len(netp), len(new_net))
        netp = eliminate_pairs(new_net)
        new_net = eliminate_a_triple_pair(netp)
    return netp

In [149]:
simple_lnet = simplify(plnet)

eatp [(Link(height=316, left=8, right=17), Link(height=317, left=8, right=21), Link(height=318, left=8, right=17), Link(height=319, left=8, right=21)), (Link(height=867, left=4, right=11), Link(height=868, left=5, right=11), Link(height=869, left=4, right=11), Link(height=870, left=5, right=11))]
removing Link(height=316, left=8, right=17) Link(height=317, left=8, right=21) Link(height=318, left=8, right=17) Link(height=319, left=8, right=21) ; adding Link(height=316, left=8, right=21) Link(height=317, left=8, right=17)
eatp [(Link(height=866, left=4, right=11), Link(height=867, left=5, right=11), Link(height=868, left=4, right=11), Link(height=869, left=5, right=11))]
removing Link(height=866, left=4, right=11) Link(height=867, left=5, right=11) Link(height=868, left=4, right=11) Link(height=869, left=5, right=11) ; adding Link(height=866, left=5, right=11) Link(height=867, left=4, right=11)
eatp []


In [150]:
''.join(follow_many(string.ascii_lowercase, simple_lnet)) == ''.join(follow_many(string.ascii_lowercase, lnet))

True

In [151]:
''.join(follow_many(string.ascii_lowercase, simple_lnet))

'zdunvslcfiqywjmkobhxtraegp'

In [152]:
''.join(follow_many(string.ascii_lowercase, lnet))

'zdunvslcfiqywjmkobhxtraegp'

In [153]:
len(simple_lnet)

9807

In [ ]:
def add_triple_pair(net, max_lines=None):
    if not max_lines:
        max_lines = max(l.right for l in net)
    a, b, c = 0, 0, 0
    while len(set((a, b, c))) != 3:
        a = random.randrange(max_lines)
        b = random.randrange(max_lines)
        c = random.randrange(max_lines)
    tp = [(min(a, b), max(a, b)), (min(b, c), max(b, c))] * 2
    
    pairs = [(l.left, l.right) for l in sorted(net)]
    i = random.randrange(len(pairs))
    print(i, tp)
    new_pairs = pairs[:i] + tp + pairs[i:]
    return pack([Link(h, l, r) for h, (l, r) in enumerate(new_pairs)])   

In [ ]:
def add_pair(net, max_lines=None):
    if not max_lines:
        max_lines = max(l.right for l in net)

    a, b = 0, 0
    while a == b:
        a = random.randrange(max_lines)
        b = random.randrange(max_lines)
    p = [(min(a, b), max(a, b))] * 2
    
    pairs = [(l.left, l.right) for l in sorted(net)]
    i = random.randrange(len(pairs))
    
    print(i, p)
    new_pairs = pairs[:i] + p + pairs[i:]
    return pack([Link(h, l, r) for h, (l, r) in enumerate(new_pairs)])   

In [ ]:
height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

In [ ]:
lnettp = simple_lnet
for _ in range(10):
    lnettp = add_pair(lnettp)
height_groups = {h: list(g) for h, g in itertools.groupby(pack(lnettp), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

In [ ]:
lnettp = simple_lnet
for _ in range(10):
    lnettp = add_triple_pair(lnettp)
height_groups = {h: list(g) for h, g in itertools.groupby(pack(lnettp), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

In [ ]:
lnettp = simple_lnet
for _ in range(10):
    lnettp = add_pair(lnettp)
for _ in range(10):
    lnettp = add_triple_pair(lnettp)
height_groups = {h: list(g) for h, g in itertools.groupby(pack(lnettp), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

In [ ]:
lnettp == pack(lnettp)

In [ ]:
lnettps = simplify(lnettp)
len(lnettps)

In [ ]:
len(simple_lnet), len(lnettp), len(lnettps)